In [1]:
!nvidia-smi

Sat May 29 05:57:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing
!ls

Mounted at /content/gdrive
/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing
ResUNetplusplus


In [3]:
%cd ./ResUNetplusplus
!ls 

/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing/ResUNetplusplus
'=1.0.7,'			       parameter_tuning.py
 bagging_approach.py		       process_image_deployment.py
 bagging_training_seed_old.py	       process_image.py
 bagging_training_seed.py	       process_image_sample.py
 boostrapping_images.ipynb	       process_image_tuning.py
 callbacks.py			       __pycache__
'Copy of execution_script.ipynb'       README.md
'Copy of TPU_execution_script.ipynb'   result
 data				       results.png
 data_generator.py		       resume_training.py
 deployment_model.py		       Resunet++.png
 execution_script.ipynb		       resunet_pp_modified.py
 files				       resunet.py
 files_before_tpu		       run_miouTesting.py
 files_kvasir_seg		       run_modified.py
 frozen_models			       run.py
 GFLOPs.ipynb			       run_sample.py
 hyperparameter_tuning.ipynb	       script_for_rafae.ipynb
 image_stiching.ipynb		       segmentation_metrics.py
 inference_script_cvc.ipynb	   

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from tensorflow.keras.optimizers import Adam, Nadam, SGD , RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from data_generator import DataGen
from unet import Unet
from resunet import ResUnet
# from m_resunet import ResUnetPlusPlus
from m_resunet_modified import ResUnetPlusPlus #daspp
# from resunet_pp_modified import ResUnetPlusPlus #sepv_conv
from metrics import dice_coef, dice_loss, miou_coef, miou_loss



In [ ]:

## Path
file_path = "files/"
model_path = "files/unet_dice_cvc2.h5"
model_name = "unet_dice_cvc2"

## Create files folder
try:
    os.mkdir("files")
except:
    pass

train_path = "new_data/CVC-ClinicDB/train/"
valid_path = "new_data/CVC-ClinicDB/valid/"

## Training
train_image_paths = glob(os.path.join(train_path, "images", "*"))
train_mask_paths = glob(os.path.join(train_path, "masks", "*"))
train_image_paths.sort()
train_mask_paths.sort()

# train_image_paths = train_image_paths[:2000]
# train_mask_paths = train_mask_paths[:2000]

## Validation
valid_image_paths = glob(os.path.join(valid_path, "images", "*"))
valid_mask_paths = glob(os.path.join(valid_path, "masks", "*"))
valid_image_paths.sort()
valid_mask_paths.sort()

In [ ]:
print(len(train_image_paths), len(train_mask_paths))

np.random.seed(0)

bts_train_image_paths = list(np.random.choice(train_image_paths , size = int(len(train_image_paths)*0.5) , replace = True))
bts_train_mask_paths = [i.replace("images","masks") for i in bts_train_image_paths] #list(np.random.choice(train_mask_paths , size = 10 , replace = True))
print(len(bts_train_image_paths), len(bts_train_mask_paths))

14880 14880
7440 7440


In [ ]:
bts_train_image_paths

[]

In [ ]:
bts_train_mask_paths

['new_data/CVC-ClinicDB/train/masks/203_12.jpg',
 'new_data/CVC-ClinicDB/train/masks/499_19.jpg',
 'new_data/CVC-ClinicDB/train/masks/464_25.jpg',
 'new_data/CVC-ClinicDB/train/masks/228_17.jpg',
 'new_data/CVC-ClinicDB/train/masks/578_18.jpg',
 'new_data/CVC-ClinicDB/train/masks/284_3.jpg',
 'new_data/CVC-ClinicDB/train/masks/67_15.jpg',
 'new_data/CVC-ClinicDB/train/masks/441_25.jpg',
 'new_data/CVC-ClinicDB/train/masks/394_24.jpg',
 'new_data/CVC-ClinicDB/train/masks/268_10.jpg']

In [ ]:
import collections

a = collections.Counter(bts_train_image_paths)
sorted(a.items(), key=lambda item: item[1], reverse = True)

[('new_data/CVC-ClinicDB/train/images/139_25.jpg', 5),
 ('new_data/CVC-ClinicDB/train/images/270_13.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/478_19.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/398_9.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/333_12.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/35_7.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/221_3.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/39_12.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/2_15.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/461_19.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/45_12.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/434_3.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/30_26.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/520_15.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/395_12.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/593_24.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/454_27.jpg', 4),
 ('new_data/CVC-ClinicDB/train/images/216_24.jpg', 4),
 ('new_data/CVC-Clin

In [ ]:
a = collections.Counter(bts_train_mask_paths)
sorted(a.items(), key=lambda item: item[1], reverse = True)

[('new_data/CVC-ClinicDB/train/masks/139_25.jpg', 5),
 ('new_data/CVC-ClinicDB/train/masks/270_13.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/478_19.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/398_9.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/333_12.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/35_7.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/221_3.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/39_12.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/2_15.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/461_19.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/45_12.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/434_3.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/30_26.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/520_15.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/395_12.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/593_24.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/454_27.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/216_24.jpg', 4),
 ('new_data/CVC-ClinicDB/train/masks/6

In [ ]:
### Script Building
%%writefile bagging_training_seed.py
import os
import numpy as np
import sys
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from tensorflow.keras.optimizers import Adam, Nadam, SGD , RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from data_generator import DataGen
from unet import Unet
from resunet import ResUnet
# from m_resunet import ResUnetPlusPlus
from m_resunet_modified import ResUnetPlusPlus #daspp
# from resunet_pp_modified import ResUnetPlusPlus #sepv_conv
from metrics import dice_coef, dice_loss, miou_coef, miou_loss


if __name__ == "__main__":
    print(sys.argv[0]) # prints python_script.py
    print(sys.argv[1]) # prints var1
    
    
    seed_value = sys.argv[1]

    ## Path
    file_path = "files/bagging/"
    model_path = "files/unet_cvc2"+str(seed_value)+".h5"
    model_name = "unet_cvc2"+str(seed_value)

    

    ## Create files folder
    try:
        os.mkdir("files/bagging")
    except:
        pass

    train_path = "new_data/CVC-ClinicDB/train/"
    valid_path = "new_data/CVC-ClinicDB/valid/"

    ## Training
    train_image_paths = glob(os.path.join(train_path, "images", "*"))
    train_mask_paths = glob(os.path.join(train_path, "masks", "*"))
    train_image_paths.sort()
    train_mask_paths.sort()

    print(len(train_image_paths), len(train_mask_paths))

    np.random.seed(int(seed_value))

    bts_train_image_paths = list(np.random.choice(train_image_paths , size = int(len(train_image_paths)*0.5) , replace = True))
    bts_train_mask_paths = [i.replace("images","masks") for i in bts_train_image_paths] #list(np.random.choice(train_mask_paths , size = 10 , replace = True))
    print(len(bts_train_image_paths), len(bts_train_mask_paths))

    # train_image_paths = train_image_paths[:2000]
    # train_mask_paths = train_mask_paths[:2000]

    ## Validation
    valid_image_paths = glob(os.path.join(valid_path, "images", "*"))
    valid_mask_paths = glob(os.path.join(valid_path, "masks", "*"))
    valid_image_paths.sort()
    valid_mask_paths.sort()

    ## Parameters
    image_size = 256
    batch_size = 4
    lr = 1e-4
    epochs = 50

    train_steps = len(bts_train_image_paths)//batch_size
    valid_steps = len(valid_image_paths)//batch_size

    print("train steps:", train_steps , "valid_steps", valid_steps)

    ## Generator
    train_gen = DataGen(image_size, bts_train_image_paths, bts_train_mask_paths, batch_size=batch_size)
    valid_gen = DataGen(image_size, valid_image_paths, valid_mask_paths, batch_size=batch_size)

    ## Unet
    arch = Unet(input_size=image_size)
    model = arch.build_model()

    optimizer = Nadam(lr)
    metrics = [Recall(), Precision(), dice_coef, MeanIoU(num_classes=2),miou_coef]
    model.compile(loss=miou_loss, optimizer=optimizer, metrics=metrics)

    csv_logger = CSVLogger(f"{file_path}{model_name}_{batch_size}_{epochs}_seed{str(seed_value)}.csv", append=False)
    checkpoint = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=5, min_lr=1e-7, verbose=1)
    callbacks = [csv_logger, checkpoint, reduce_lr]

    model.fit_generator(train_gen,
            validation_data=valid_gen,
            steps_per_epoch=train_steps,
            validation_steps=valid_steps,
            epochs=epochs,
            callbacks=callbacks)

Overwriting bagging_training_seed.py


In [ ]:
!python3 bagging_training_seed.py 0

2021-05-24 05:11:55.828729: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
bagging_training_seed.py
0
14880 14880
7440 7440
bts_train_image_paths
[('new_data/CVC-ClinicDB/train/images/139_25.jpg', 5), ('new_data/CVC-ClinicDB/train/images/270_13.jpg', 4), ('new_data/CVC-ClinicDB/train/images/478_19.jpg', 4), ('new_data/CVC-ClinicDB/train/images/398_9.jpg', 4), ('new_data/CVC-ClinicDB/train/images/333_12.jpg', 4)]
bts_train_mask_paths
[('new_data/CVC-ClinicDB/train/masks/139_25.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/270_13.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/478_19.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/398_9.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/333_12.jpg', 4)]
train steps: 1860 valid_steps 15
2021-05-24 05:15:22.091106: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-24 05:15:22.091979: I tensorflow/stream_executor/p

In [ ]:
!python3 bagging_training_seed.py 1

2021-05-24 08:23:47.277667: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
bagging_training_seed.py
1
14880 14880
7440 7440
bts_train_image_paths
[('new_data/CVC-ClinicDB/train/images/367_18.jpg', 5), ('new_data/CVC-ClinicDB/train/images/142_27.jpg', 5), ('new_data/CVC-ClinicDB/train/images/25_23.jpg', 5), ('new_data/CVC-ClinicDB/train/images/238_7.jpg', 4), ('new_data/CVC-ClinicDB/train/images/54_14.jpg', 4)]
bts_train_mask_paths
[('new_data/CVC-ClinicDB/train/masks/367_18.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/142_27.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/25_23.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/238_7.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/54_14.jpg', 4)]
train steps: 1860 valid_steps 15
2021-05-24 08:23:49.290992: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-24 08:23:49.291906: I tensorflow/stream_executor/platf

In [ ]:
!python3 bagging_training_seed.py 2

2021-05-25 05:35:33.220898: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
bagging_training_seed.py
2
14880 14880
7440 7440
bts_train_image_paths
[('new_data/CVC-ClinicDB/train/images/277_24.jpg', 4), ('new_data/CVC-ClinicDB/train/images/80_10.jpg', 4), ('new_data/CVC-ClinicDB/train/images/113_25.jpg', 4), ('new_data/CVC-ClinicDB/train/images/107_12.jpg', 4), ('new_data/CVC-ClinicDB/train/images/34_10.jpg', 4)]
bts_train_mask_paths
[('new_data/CVC-ClinicDB/train/masks/277_24.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/80_10.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/113_25.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/107_12.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/34_10.jpg', 4)]
train steps: 1860 valid_steps 15
2021-05-25 05:39:09.643528: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-25 05:39:09.644619: I tensorflow/stream_executor/pla

In [ ]:
!python3 bagging_training_seed.py 3

2021-05-25 08:49:12.251857: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
bagging_training_seed.py
3
14880 14880
7440 7440
bts_train_image_paths
[('new_data/CVC-ClinicDB/train/images/38_14.jpg', 6), ('new_data/CVC-ClinicDB/train/images/472_13.jpg', 5), ('new_data/CVC-ClinicDB/train/images/495_7.jpg', 5), ('new_data/CVC-ClinicDB/train/images/489_13.jpg', 4), ('new_data/CVC-ClinicDB/train/images/167_26.jpg', 4)]
bts_train_mask_paths
[('new_data/CVC-ClinicDB/train/masks/38_14.jpg', 6), ('new_data/CVC-ClinicDB/train/masks/472_13.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/495_7.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/489_13.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/167_26.jpg', 4)]
train steps: 1860 valid_steps 15
2021-05-25 08:49:14.229708: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-25 08:49:14.230679: I tensorflow/stream_executor/pla

In [ ]:
!python3 bagging_training_seed.py 4

2021-05-27 05:33:22.772196: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
bagging_training_seed.py
4
14880 14880
7440 7440
bts_train_image_paths
[('new_data/CVC-ClinicDB/train/images/37_27.jpg', 5), ('new_data/CVC-ClinicDB/train/images/2_30.jpg', 5), ('new_data/CVC-ClinicDB/train/images/43_3.jpg', 5), ('new_data/CVC-ClinicDB/train/images/314_5.jpg', 4), ('new_data/CVC-ClinicDB/train/images/462_24.jpg', 4)]
bts_train_mask_paths
[('new_data/CVC-ClinicDB/train/masks/37_27.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/2_30.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/43_3.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/314_5.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/462_24.jpg', 4)]
train steps: 1860 valid_steps 15
2021-05-27 05:33:24.636006: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-05-27 05:33:24.664778: I tensorflow/stream_executor/cuda/c

In [ ]:
!python3 bagging_training_seed.py 5

2021-05-27 08:11:03.965890: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
bagging_training_seed.py
5
14880 14880
7440 7440
bts_train_image_paths
[('new_data/CVC-ClinicDB/train/images/138_17.jpg', 5), ('new_data/CVC-ClinicDB/train/images/192_18.jpg', 4), ('new_data/CVC-ClinicDB/train/images/346_29.jpg', 4), ('new_data/CVC-ClinicDB/train/images/345_15.jpg', 4), ('new_data/CVC-ClinicDB/train/images/122_15.jpg', 4)]
bts_train_mask_paths
[('new_data/CVC-ClinicDB/train/masks/138_17.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/192_18.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/346_29.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/345_15.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/122_15.jpg', 4)]
train steps: 1860 valid_steps 15
2021-05-27 08:11:05.757701: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-05-27 08:11:05.787002: I tensorflow/stream_exe

In [5]:
!python3 bagging_training_seed.py 6

2021-05-29 06:02:27.259119: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
bagging_training_seed.py
6
14880 14880
7440 7440
bts_train_image_paths
[('new_data/CVC-ClinicDB/train/images/517_15.jpg', 6), ('new_data/CVC-ClinicDB/train/images/145_5.jpg', 5), ('new_data/CVC-ClinicDB/train/images/519_4.jpg', 5), ('new_data/CVC-ClinicDB/train/images/372_5.jpg', 5), ('new_data/CVC-ClinicDB/train/images/358_14.jpg', 5)]
bts_train_mask_paths
[('new_data/CVC-ClinicDB/train/masks/517_15.jpg', 6), ('new_data/CVC-ClinicDB/train/masks/145_5.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/519_4.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/372_5.jpg', 5), ('new_data/CVC-ClinicDB/train/masks/358_14.jpg', 5)]
train steps: 1860 valid_steps 15
2021-05-29 06:02:29.098308: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-05-29 06:02:29.124549: I tensorflow/stream_executor/

In [6]:
!python3 bagging_training_seed.py 7

2021-05-29 09:15:46.802672: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
bagging_training_seed.py
7
14880 14880
7440 7440
bts_train_image_paths
[('new_data/CVC-ClinicDB/train/images/62_15.jpg', 4), ('new_data/CVC-ClinicDB/train/images/470_26.jpg', 4), ('new_data/CVC-ClinicDB/train/images/10_24.jpg', 4), ('new_data/CVC-ClinicDB/train/images/34_4.jpg', 4), ('new_data/CVC-ClinicDB/train/images/223_17.jpg', 4)]
bts_train_mask_paths
[('new_data/CVC-ClinicDB/train/masks/62_15.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/470_26.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/10_24.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/34_4.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/223_17.jpg', 4)]
train steps: 1860 valid_steps 15
2021-05-29 09:15:48.566791: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-05-29 09:15:48.597244: I tensorflow/stream_executor/cu

In [7]:
!python3 bagging_training_seed.py 8

2021-05-29 12:12:50.947271: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
bagging_training_seed.py
8
14880 14880
7440 7440
bts_train_image_paths
[('new_data/CVC-ClinicDB/train/images/216_1.jpg', 6), ('new_data/CVC-ClinicDB/train/images/308_7.jpg', 4), ('new_data/CVC-ClinicDB/train/images/124_16.jpg', 4), ('new_data/CVC-ClinicDB/train/images/315_20.jpg', 4), ('new_data/CVC-ClinicDB/train/images/425_18.jpg', 4)]
bts_train_mask_paths
[('new_data/CVC-ClinicDB/train/masks/216_1.jpg', 6), ('new_data/CVC-ClinicDB/train/masks/308_7.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/124_16.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/315_20.jpg', 4), ('new_data/CVC-ClinicDB/train/masks/425_18.jpg', 4)]
train steps: 1860 valid_steps 15
2021-05-29 12:12:52.745402: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-05-29 12:12:52.775581: I tensorflow/stream_executo